### Preparing dataset_subset_XXX.json 

In [1]:
import sys
sys.path.append('/home/nakamura/project/python3_selfsequential/vg')
import visual_genome.local as vg
dir = '/mnt/poplin/share/dataset/visualgenome'
all_images = vg.get_all_image_data(dir)
all_discriptions = vg.get_all_region_descriptions(dir)

In [ ]:
all_discriptions[36025]

In [2]:
import sys

import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from visual_genome import api
from PIL import Image as PIL_Image
import requests
import pdb
import numpy as np
import json

In [3]:
f = open('/home/nakamura/project/self_seqential/data/dataset_coco.json', 'r')
info_base = json.load(f)

info_vg = {}
info_vg['dataset'] = 'vg'
info_vg['images'] = []

info_coco = {}
info_coco['dataset'] = 'coco'
info_coco['images'] = []

info_coco_vg = {}
info_coco_vg['dataset'] = 'coco and vg'
info_coco_vg['images'] = []

info_vg_larger = {}
info_vg_larger['dataset'] = 'vg_larger'
info_vg_larger['images'] = []

順番が適当な場合

In [ ]:
import copy
for i, img in enumerate(info_base['images']):
    for j , vg_img in enumerate(all_images):
        if img['cocoid'] == vg_img.coco_id:
            img_info = {}
            img_info['filepath'] = img['filepath']
            img_info['sentids'] = img['sentids']
            img_info['filename'] = img['filename']
            img_info['imgid'] = img['imgid']
            img_info['split'] = img['split']
            img_info['cocoid'] = img['cocoid']
            
            sentenses_coco = img['sentences']
            
            sentenses_vg = []
            for region in all_discriptions[j]:
                sent_info = {}
                sent_info['imgid'] = i
                sent_info['raw'] = region.phrase
                pdb.set_trace()
                sent_info['tokens'] = region.phrase.split()
                sentenses_vg.append(sent_info)
                
            sentenses = sentenses_vg + sentenses_coco
            img_info_coco_vg = copy.copy(img_info)
            img_info_coco = copy.copy(img_info)
            img_info_vg = copy.copy(img_info)
            
            img_info_coco_vg['sentences'] = sentenses
            info_coco_vg['images'].append(img_info_coco_vg)
            
            img_info_vg['sentences'] = sentenses_vg
            info_vg['images'].append(img_info_vg)
            
            img_info_coco['sentences'] = sentenses_coco
            info_coco['images'].append(img_info_coco)
            
            print(len(img_info_coco_vg['sentences']))
            print(len(img_info_coco['sentences']))
            print(len(img_info_vg['sentences']))
            
            break

Visual genome の領域の順番が決まっている場合．

In [ ]:
all_set_regions_nodup = json.load(open('/mnt/workspace2018/nakamura/selfsequential/data/marged_set_larger_nodup.json', 'r'))
# all_set_regions_nodup = json.load(open('/mnt/workspace2018/nakamura/selfsequential/data/marged_set_allvg_07.json', 'r'))

In [5]:
all_set_regions_nodup = json.load(open('/mnt/workspace2018/nakamura/selfsequential/data/marged_set_larger_morethan3.json', 'r'))
all_set_regions_nodup_flg = all_set_regions_nodup['flg']
all_set_regions_nodup = all_set_regions_nodup['maged_set']

In [7]:
cor_tables = json.load(open('/mnt/workspace2018/nakamura/selfsequential/data/cor_tables.json', 'r'))

In [8]:
# cocoとの共通の場合
import copy

max_att_length = 88
imgs = info_base['images']
for num, table in enumerate(cor_tables):
    i = table[0]
    j = table[1]
    img = imgs[i]
    vg_image = all_images[j]

    img_info = {}
    img_info['filepath'] = img['filepath']
    img_info['sentids'] = img['sentids']
    img_info['filename'] = img['filename']
    img_info['imgid'] = img['imgid']
    img_info['split'] = img['split']
    img_info['cocoid'] = img['cocoid']
    
    sentenses_coco = img['sentences']

    sentenses_vg = []
    for k_num, key in enumerate(all_set_regions_nodup[num].keys()):
        if k_num < max_att_length:
#             print(k_num)
            region_list = all_set_regions_nodup[num][key]

            for r_num in region_list:
                region = all_discriptions[j][r_num]
                sent_info = {}
                sent_info['imgid'] = i
                sent_info['raw'] = region.phrase.lower()
                sent_info['tokens'] = region.phrase.lower().split()
                sentenses_vg.append(sent_info)
        
    sentenses = sentenses_vg + sentenses_coco

    img_info_coco = copy.copy(img_info)
    img_info_vg_larger = copy.copy(img_info)

    img_info_vg_larger['sentences'] = sentenses_vg
    info_vg_larger['images'].append(img_info_vg_larger)

    img_info_coco['sentences'] = sentenses_coco
    info_coco['images'].append(img_info_coco)
    
print(len(img_info_coco['sentences']))
print(len(img_info_vg_larger['sentences']))

5
45


In [10]:
json.dump(info_vg_larger, open('/mnt/workspace2018/nakamura/selfsequential/data/dataset_allvg07_larger_morethan3gt.json', 'w'))

In [ ]:
# Visual genome のsplit の決定
val_test_split_id = np.random.permutation(np.arange(len(all_images)))

In [ ]:
max_region_num = 0
for num in range(len(all_set_regions_nodup)):
    region_num = len(all_set_regions_nodup[num].keys()) + 1
    if max_region_num < region_num:
        max_region_num = region_num
print(max_region_num)

In [ ]:
#Visual genomeのみの場合
vg_dataset = json.load(open('/mnt/workspace2018/nakamura/selfsequential/data/dataset_subset_vg_larger_all.json'))
image_dir = '/mnt/workspace2018/nakamura/vg_images/VG_100K'

import os
import copy

imgs = info_base['images']
for num, vg_image in enumerate(all_images):
    
    if os.path.exists(image_dir + '/' + str(vg_image.id) + '.jpg'):
#         I = skimage.io.imread(image_dir + '/' + str(vg_image.id) + '.jpg')
        filepath = 'VG_100K'
    else:
#         I = skimage.io.imread(image_dir + '_2/' + str(vg_image.id) + '.jpg')
        filepath = 'VG_100K_2'

    img_info = {}
    img_info['filepath'] = filepath
    img_info['sentids'] = num
    img_info['filename'] = str(vg_image.id)
    img_info['imgid'] = str(vg_image.id)
    if num in val_test_split_id[:5000]:
        img_info['split'] = 'val'
    elif num in val_test_split_id[5000:10000]:
        img_info['split'] = 'test'
    else:
        img_info['split'] = 'train'
    img_info['cocoid'] = str(vg_image.id)

    sentenses_vg = []
    for k_num, key in enumerate(all_set_regions_nodup[num].keys()):
        region_list = all_set_regions_nodup[num][key]
        
        region = all_discriptions[num][int(key)]
        sent_info = {}
        sent_info['imgid'] = num
        sent_info['raw'] = region.phrase.lower()
        sent_info['tokens'] = region.phrase.lower().split()
        sentenses_vg.append(sent_info)
        
        for r_num in region_list:
            region = all_discriptions[num][r_num]
            sent_info = {}
            sent_info['imgid'] = num
            sent_info['raw'] = region.phrase.lower()
            sent_info['tokens'] = region.phrase.lower().split()
            sentenses_vg.append(sent_info)
        
    sentenses = sentenses_vg

    img_info_vg = copy.copy(img_info)
    img_info_vg['sentences'] = sentenses_vg
    info_vg['images'].append(img_info_vg)
    
    if num % 1000 == 0:
        print('processing %d/%d (%.2f%% done)' % (num, len(all_images), num*100.0/len(all_images)))

In [ ]:
json.dump(info_vg, open('/mnt/workspace2018/nakamura/selfsequential/data/dataset_allvg07_larger.json', 'w'))

In [ ]:
json.dump(info_vg_larger, open('/mnt/workspace2018/nakamura/selfsequential/data/dataset_subset_vg_larger.json', 'w'))
json.dump(info_coco, open('/mnt/workspace2018/nakamura/selfsequential/data/dataset_subset_coco_larger.json', 'w'))

### make json and labelfile

In [11]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import json
import argparse
from random import shuffle, seed
import string
# non-standard dependencies:
import h5py
import numpy as np
import torch
import torchvision.models as models
import skimage.io
from PIL import Image
import pdb
import prepro_labels as labels

import easydict
# args = easydict.EasyDict({
#         "input_json": '/home/nakamura/project/self_seqential/data/dataset_subset_coco_and_vg_.json',
#         "output_json": '/mnt/poplin/share/dataset/MSCOCO/cocotalk_subset_mscoco_and_vg_.json',
#         "output_h5": '/mnt/poplin/share/dataset/MSCOCO/cocotalk_subset_mscoco_and_vg_',
#         "image_root": '',
#         "max_length": 30,
#         "word_count_threshold": 5
# })

args = easydict.EasyDict({
        "input_json": '/home/nakamura/project/self_seqential/data/dataset_subset_coco_and_vg_.json',
        "output_json": '/mnt/poplin/share/dataset/MSCOCO/sample.json',
        "output_h5": '/mnt/poplin/share/dataset/MSCOCO/sample',
        "image_root": '',
        "max_length": 30,
        "word_count_threshold": 5
})

# args_vg = easydict.EasyDict({
#         "input_json": '/home/nakamura/project/self_seqential/data/dataset_subset_vg_.json',
#         "output_json": '/mnt/poplin/share/dataset/MSCOCO/cocotalk_subset_vg_.json',
#         "output_h5": '/mnt/poplin/share/dataset/MSCOCO/cocotalk_subset_vg_',
#         "image_root": '',
#         "max_length": 30,
#         "word_count_threshold": 5
# })

args_vg = easydict.EasyDict({
        "input_json": '/mnt/workspace2018/nakamura/selfsequential/data/dataset_subset_vg_larger_all.json',
        "output_json": '/mnt/poplin/share/dataset/MSCOCO/cocotalk_subset_vg_larger_all.json',
        "output_h5": '/mnt/poplin/share/dataset/MSCOCO/cocotalk_subset_vg_larger_all',
        "image_root": '',
        "max_length": 30,
        "word_count_threshold": 5
})

# args_vg = easydict.EasyDict({
#         "input_json": '/mnt/workspace2018/nakamura/selfsequential/data/dataset_allvg07_larger.json',
#         "output_json": '/mnt/workspace2018/nakamura/selfsequential/data/cocotalk_allvg07_larger.json',
#         "output_h5": '/mnt/workspace2018/nakamura/selfsequential/data/cocotalk_allvg07_larger',
#         "image_root": '',
#         "max_length": 30,
#         "word_count_threshold": 5
# })

args_mscoco = easydict.EasyDict({
        "input_json": '/mnt/workspace2018/nakamura/selfsequential/data/dataset_subset_coco_larger.json',
        "output_json": '/mnt/poplin/share/dataset/MSCOCO/cocotalk_subset_mscoco_larger.json',
        "output_h5": '/mnt/poplin/share/dataset/MSCOCO/cocotalk_subset_mscoco_larger',
        "image_root": '',
        "max_length": 30,
        "word_count_threshold": 5
})

args_sew = easydict.EasyDict({
        "input_json": '/mnt/poplin/share/dataset/simple_english_wikipedia/dataset_sew_unk2_l0.json',
        "output_json": '/mnt/poplin/share/dataset/simple_english_wikipedia/sew_talk.json',
        "output_h5": '/mnt/poplin/share/dataset/simple_english_wikipedia/sew_talk',
        "image_root": '',
        "max_length": 30,
        "word_count_threshold": 0
})

args_3gt = easydict.EasyDict({
        "input_json": '/mnt/workspace2018/nakamura/selfsequential/data/dataset_allvg07_larger_morethan3gt.json',
        "output_json": '/mnt/workspace2018/nakamura/selfsequential/data/cocotalk_subset_vg_larger_morethan3gt.json',
        "output_h5": '/mnt/workspace2018/nakamura/selfsequential/data/cocotalk_subset_vg_larger_morethan3gt_label_all.h5',
        "image_root": '',
        "max_length": 30,
        "word_count_threshold": 0
})

In [ ]:
info_dataset = json.load(open('/mnt/workspace2018/nakamura/selfsequential/data/dataset_subset_vg_larger.json', 'r'))

In [ ]:
info_json = json.load(open('/mnt/poplin/share/dataset/MSCOCO/cocotalk_subset_mscoco_larger.json', 'r'))

In [ ]:
itw = info_json['ix_to_word']

In [ ]:
info_dataset

In [ ]:
st = ''
for ix in array:
    if ix > 0:
        st += itw[str(ix)]
        st += ' '
print(st)

In [ ]:
info_json['images'][37]

In [ ]:
count = 0
for info in info_dataset['images'][37]['sentences']:
    print(count, info['raw'])
    count += 1

In [ ]:
# all_dets = json.load(open('/mnt/workspace2018/nakamura/selfsequential/data/region_dets_larger.json', 'r'))
all_dets = json.load(open('/mnt/workspace2018/nakamura/selfsequential/data/region_dets_allvg_07.json', 'r'))

In [ ]:
count = 0
for info in all_dets[37]:
    print(count, info)
    count += 1
# all_dets[37]

In [ ]:
for i, img in enumerate(all_images):
    if img.coco_id == 480985:
        print(all_discriptions[i][29])

In [ ]:
# Preparing vocab

imgs = json.load(open(args['input_json'], 'r'))
imgs = imgs['images']

imgs_vg = json.load(open(args_vg['input_json'], 'r'))
imgs_vg = imgs_vg['images']

imgs_coco = json.load(open(args_mscoco['input_json'], 'r'))
imgs_coco = imgs_coco['images']

seed(123) # make reproducible

# create the vocab
vocab = labels.build_vocab(imgs, args)
itow = {i+1:w for i,w in enumerate(vocab)} # a 1-indexed vocab translation table
wtoi = {w:i+1 for i,w in enumerate(vocab)} # inverse table

In [12]:
info = json.load(open('/mnt/poplin/share/dataset/MSCOCO/cocotalk_subset_vg_larger.json', 'r'))
ix_to_word = info['ix_to_word']

In [13]:
wtoi = {}
for k in ix_to_word.keys():
    wtoi[ix_to_word[k]] = k

In [14]:
# imgs_vg = json.load(open(args_vg['input_json'], 'r'))
# imgs_vg = imgs_vg['images']
imgs_vg = json.load(open(args_3gt['input_json'], 'r'))
imgs_vg = imgs_vg['images']

In [22]:
# if you use more than 3 gt, selsect use caption
info_img = info['images']
info_morethan3gt = {}
info_img_morethan3gt = []
for i in range(len(info_img)):
    element = {}
    if info_img[i]['split'] == 'train' or info_img[i]['split'] == 'restval':
        if all_set_regions_nodup_flg[i] == 1:
            info_img_morethan3gt.append(info_img[i])
        else:
            info_img[i]['split'] = 'nouse'
            info_img_morethan3gt.append(info_img[i])
    else:
        info_img_morethan3gt.append(info_img[i])
info_morethan3gt['images'] = info_img_morethan3gt
info_morethan3gt['ix_to_word'] = ix_to_word

In [23]:
json.dump(info_morethan3gt, open('/mnt/poplin/share/dataset/MSCOCO/cocotalk_subset_vg_larger_morethan3gt.json', 'w'))

In [ ]:
imgs_sew = json.load(open(args_sew['input_json'], 'r'))
imgs_sew = imgs_sew['images']

In [15]:
# L, label_start_ix, label_end_ix, label_length = labels.encode_captions(imgs, args, wtoi)

# for img in imgs_coco:
#     img['final_captions'] = []
#     for sent in img['sentences']:
#         txt = sent['tokens']
#         caption = [w if w in wtoi.keys() else 'UNK' for w in txt]
#         img['final_captions'].append(caption)

# L_coco, label_start_ix_coco, label_end_ix_coco, label_length_coco = labels.encode_captions(imgs_coco, args_mscoco, wtoi)

for num, img in enumerate(imgs_vg):
    img['final_captions'] = []
    for sent_num in range(len(img['sentences'])):
        sent = img['sentences'][sent_num]
        txt = sent['tokens']
#         caption = [w if w in wtoi.keys() else 'UNK' for w in txt]
        caption = [w.lower() if w.lower() in wtoi.keys() else 'UNK' for w in txt]
        if len(txt) == 0:
            print(caption)
            caption = ['UNK']
            print(caption)
#         print(caption)
#         pdb.set_trace()
        img['final_captions'].append(caption)
    
    if num % 1000 == 0:
        print('processing %d/%d (%.2f%% done)' % (num, len(imgs_vg), num*100.0/len(imgs_vg)))

L_vg, label_start_ix_vg, label_end_ix_vg, label_length_vg = labels.encode_captions(imgs_vg, args_vg, wtoi)

for img in imgs_vg:
    if len(img['sentences']) != len(img['final_captions']):
        pdb.set_trace()

processing 0/51197 (0.00% done)
processing 1000/51197 (1.95% done)
processing 2000/51197 (3.91% done)
processing 3000/51197 (5.86% done)
processing 4000/51197 (7.81% done)
processing 5000/51197 (9.77% done)
processing 6000/51197 (11.72% done)
processing 7000/51197 (13.67% done)
processing 8000/51197 (15.63% done)
[]
['UNK']
processing 9000/51197 (17.58% done)
processing 10000/51197 (19.53% done)
processing 11000/51197 (21.49% done)
processing 12000/51197 (23.44% done)
processing 13000/51197 (25.39% done)
[]
['UNK']
processing 14000/51197 (27.35% done)
processing 15000/51197 (29.30% done)
processing 16000/51197 (31.25% done)
processing 17000/51197 (33.21% done)
processing 18000/51197 (35.16% done)
processing 19000/51197 (37.11% done)
processing 20000/51197 (39.06% done)
processing 21000/51197 (41.02% done)
processing 22000/51197 (42.97% done)
processing 23000/51197 (44.92% done)
processing 24000/51197 (46.88% done)
processing 25000/51197 (48.83% done)
processing 26000/51197 (50.78% done

In [ ]:
L_vg[9]

In [ ]:
for img in imgs_sew:
    img['final_captions'] = []
#     pdb.set_trace()
    for sent_num in range(len(img['sentences'])):
        sent = img['sentences'][sent_num]
        txt = sent['tokens']
        caption = [w if w in wtoi.keys() else 'UNK' for w in txt]
        if len(txt) == 0:
            print(caption)
            caption = ['UNK']
            print(caption)
        img['final_captions'].append(caption)

L_sew, label_start_ix_sew, label_end_ix_sew, label_length_sew = labels.encode_captions(imgs_sew, args_sew, wtoi)

In [ ]:
def make_labelfile(L, label_start_ix, label_end_ix, label_length, params, imgs):
    # create output h5 file
    N = len(imgs)
    f_lb = h5py.File(params['output_h5']+'_label.h5', "w")
    f_lb.create_dataset("labels", dtype='uint32', data=L)
    f_lb.create_dataset("label_start_ix", dtype='uint32', data=label_start_ix)
    f_lb.create_dataset("label_end_ix", dtype='uint32', data=label_end_ix)
    f_lb.create_dataset("label_length", dtype='uint32', data=label_length)
    f_lb.close()
    print('ok!!!')

In [16]:
def make_json(itow, imgs, params):
    out = {}
    out['ix_to_word'] = itow # encode the (1-indexed) vocab
    out['images'] = []
    for i,img in enumerate(imgs):
        jimg = {}
        jimg['split'] = img['split']
        if 'filename' in img: jimg['file_path'] = os.path.join(img['filepath'], img['filename']) # copy it over, might need
        if 'cocoid' in img: jimg['id'] = img['cocoid'] # copy over & mantain an id, if present (e.g. coco ids, useful)

        out['images'].append(jimg)

    json.dump(out, open(params['output_json'], 'w'))
    print('wrote ', params['output_json'])

In [ ]:
# make_labelfile(L, label_start_ix, label_end_ix, label_length, args, imgs)
# make_labelfile(L_coco, label_start_ix_coco, label_end_ix_coco, label_length_coco, args_mscoco, imgs_coco)
# make_labelfile(L_vg, label_start_ix_vg, label_end_ix_vg, label_length_vg, args_vg, imgs_vg)
make_labelfile(L_sew, label_start_ix_sew, label_end_ix_sew, label_length_sew, args_sew, imgs_sew)

In [ ]:
for i in range(len(imgs_vg)):
    if len(imgs_vg[i]['final_captions']) != len(info_dataset['images'][i]['sentences']):
        print(len(imgs_vg[i]['final_captions']))
        print(len(info_dataset['images'][i]['sentences']))
        pdb.set_trace()

In [17]:
# label for each region
max_region_num = 88
label_start_ix_er = np.zeros((len(label_start_ix_vg), max_region_num))
label_end_ix_er = np.zeros((len(label_end_ix_vg), max_region_num))

Len = 1
for num in range(len(label_start_ix_vg)):
    for i, key in enumerate(all_set_regions_nodup[num].keys()):
#         if i < 36:
        r_len = len(all_set_regions_nodup[num][key])
        label_start_ix_er[num, i] += Len
        Len += r_len
        label_end_ix_er[num, i] += Len - 1
    if np.max(label_end_ix_er[num]) != label_end_ix_vg[num]:
        pdb.set_trace()
print(label_start_ix_er[num])
print(label_end_ix_er[num])
print(label_start_ix_vg[num])
print(label_end_ix_vg[num])

[2204483. 2204484. 2204485. 2204486. 2204487. 2204488. 2204489. 2204490.
 2204491. 2204492. 2204493. 2204494. 2204495. 2204496. 2204497. 2204498.
 2204499. 2204500. 2204501. 2204502. 2204503. 2204504. 2204505. 2204506.
 2204507. 2204508. 2204509. 2204510. 2204511. 2204512. 2204513. 2204514.
 2204515. 2204516. 2204517. 2204518. 2204519. 2204520. 2204521. 2204522.
 2204523. 2204524. 2204525. 2204526. 2204527.       0.       0.       0.
       0.       0.       0.       0.       0.       0.       0.       0.
       0.       0.       0.       0.       0.       0.       0.       0.
       0.       0.       0.       0.       0.       0.       0.       0.
       0.       0.       0.       0.       0.       0.       0.       0.
       0.       0.       0.       0.       0.       0.       0.       0.]
[2204483. 2204484. 2204485. 2204486. 2204487. 2204488. 2204489. 2204490.
 2204491. 2204492. 2204493. 2204494. 2204495. 2204496. 2204497. 2204498.
 2204499. 2204500. 2204501. 2204502. 2204503. 2204

In [18]:
N = len(imgs_vg)
# f_lb = h5py.File('/mnt/workspace2018/nakamura/selfsequential/data/cocotalk_allvg07_larger_label.h5', 'w')
# f_lb = h5py.File('/mnt/poplin/share/dataset/MSCOCO/cocotalk_subset_vg_larger_label_all.h5', "w")
f_lb = h5py.File('/mnt/poplin/share/dataset/MSCOCO/cocotalk_subset_vg_larger_morethan3gt_label_all.h5', "w")
f_lb.create_dataset("labels", dtype='uint32', data=L_vg)
f_lb.create_dataset("label_start_ix", dtype='uint32', data=label_start_ix_vg)
f_lb.create_dataset("label_end_ix", dtype='uint32', data=label_end_ix_vg)
f_lb.create_dataset("label_length", dtype='uint32', data=label_length_vg)
f_lb.create_dataset("label_start_ix_er", dtype='uint32', data=label_start_ix_er)
f_lb.create_dataset("label_end_ix_er", dtype='uint32', data=label_end_ix_er)
f_lb.close()

In [ ]:
for i, img in enumerate(imgs_sew):
    img['filename'] = str(i)
    img['cocoid'] = str(i)
    img['filepath'] = str(i)
    img['imgid'] = str(i)
    img['sentids'] = [i]
    if (i+1) % 10 == 0:
        img['split'] = 'val'
    else:
        img['split'] = 'train'

In [ ]:
imgs_sew[1]

In [ ]:
# make_json(itow, imgs, args)
# make_json(itow, imgs_coco, args_mscoco)
make_json(ix_to_word, imgs_vg, args_vg)
# make_json(itow, imgs_sew, args_sew)

In [ ]:
info = json.load(open('/mnt/workspace2018/nakamura/IAPR/dataset_iapr.json'))

In [ ]:
len(info['ix_to_word'])

In [ ]:
ix_list = []
for ix in range(len(info['images'])):
    try:
        A = np.load(os.path.join('/mnt/workspace2018/nakamura/IAPR/features', str(info['images'][ix]['id']) + '.npz' ))
    except:
        print(info['images'][ix]['id'])
        ix_list.append(ix)

In [ ]:
for i in range(len(ix_list)):
    info['images'].pop(ix_list[len(ix_list)-1-i])

json.dump(info, open('/mnt/workspace2018/nakamura/IAPR/iapr_talk.json', 'w'))

In [ ]:
info = json.load(open('/mnt/workspace2018/nakamura/IAPR/iapr_talk_mod.json'))
# info_coco = json.load(open('/mnt/poplin/share/dataset/MSCOCO/cocotalk_subset_mscoco.json'))
import h5py
h5_label_file = h5py.File('/mnt/workspace2018/nakamura/IAPR/iaprtalk_mod_label.h5', 'r', driver='core')

In [ ]:
labels = h5_label_file['labels']
label_start_ix = h5_label_file['label_start_ix'][:]
label_end_ix = h5_label_file['label_end_ix'][:]
ix_to_word = info['ix_to_word']

In [ ]:
print(len(labels), len(label_start_ix), len(label_end_ix))

In [ ]:
def generate_sentense(label):
    sentense = ''
    for word in label:
        if word != 0:
            sentense += ix_to_word[str(word)] + ' '
    print(sentense)

In [ ]:
generate_sentense(labels[1])

In [ ]:
len(info['images'])

In [ ]:
info['images'][0]

In [ ]:
info_coco['images'][0]

In [ ]:
info_dataset['images'][0]['sentences']

In [ ]:
import os
from os import listdir
dir = '/mnt/workspace2018/nakamura/IAPR/features/'
imgs = []
for i ,img in enumerate(info['images']):
    path = dir
    img_list = os.listdir(path)
    if img['id'] + '.npz' in img_list:
        img_ = copy.copy(img)
        imgs.append(img_)

print(len(info['images']))
print(len(imgs))

In [ ]:
print(len(info['images']))
print(len(imgs))

In [ ]:
info['images'] = imgs
json.dump(info, open('/mnt/workspace2018/nakamura/IAPR/dataset_iapr_mod.json', 'w'))

In [ ]:
sys.path.append("coco-caption")
info = json.load(open('/home/nakamura/project/self_seqential/coco-caption/annotations/captions_val2014.json'))

In [ ]:
info.keys()

In [ ]:
info['info']

In [ ]:
info['images'][0]

In [ ]:
info['licenses'][0]

In [ ]:
info['type']

In [ ]:
info['annotations'][:10]